# Automatic parameter sweeping & result visualizaiton
This notebook demonstrates our ability to sweep over a set of parameters for markets and pricing models. The final results are displayed via a [parallel coordinates plot](https://arxiv.org/abs/1705.00368), which illustrates how each parameter influences a final metric, such as net fees earned.

In [ ]:
import os, sys

#if os.path.exists('analysis'):
#    os.system('rm -rf analysis')
#!git clone https://github.com/element-fi/analysis.git && cd analysis && git fetch && git checkout consistency_test && cd ..
#
#sys.path.insert(1, os.getcwd())

In [ ]:
import time

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from sim import YieldSimulator, ElementPricingModel, Market, YieldSpacev2PricingModel, YieldSpacev2MinFeePricingModel

In [ ]:
seed = 1234
rng = np.random.default_rng(seed)

config = {
    'min_fee': 0.,
    'max_fee': 0.5,
    't_min': 0.001,
    't_max': 1.0,
    'base_asset_price': 55000., # aka market price
    'min_target_liquidity': 100000.,
    'max_target_liquidity': 10000000.,
    'min_target_volume': 2e5,
    'max_target_volume': 6e5,
    'min_pool_apy': 0.5,
    'max_pool_apy': 20.0,
    'min_vault_age': 0.,
    'max_vault_age': 2.,
    'min_vault_apy': 0.,
    'max_vault_apy': 10.,
    'precision': None,
    'pricing_model_name': 'YieldSpacev2',
    'tokens': ['base', 'fyt'],
    'trade_direction': 'out',
    'days_until_maturity': 90,
    'num_trading_days': 50, # should be <= days_until_maturity
    'rng': rng, # random number generator
    'verbose': False,
}

In [ ]:
start_time = time.time()
num_runs = 0

simulator = YieldSimulator(**config)
simulator.set_random_variables()

pricing_models = ['Element', 'YieldSpacev2', 'YieldSpacev2MinFee']
vault_ages = [0.1, 0.3, 0.5]
init_pool_apys = [1, 5, 20]
vault_apys = [[i,]*config['num_trading_days'] for i in [3, 5, 9]] # multiple constant vault apy

for pricing_model in pricing_models:
    for vault_age in vault_ages:
        for vault_apy in vault_apys:
            for init_pool_apy in init_pool_apys:
                override_dict = {
                    'pricing_model_name': pricing_model,
                    'vault_apy': vault_apy,
                    'init_vault_age': vault_age,
                    'init_pool_apy': init_pool_apy,
                }
                simulator.run_simulation(override_dict)
                num_runs += 1

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
# construct simulation dataframe output
trades = pd.DataFrame.from_dict(simulator.analysis_dict)

# calculate derived variables across runs
trades['time_diff'] = trades.time_until_end.diff()
trades['time_diff_shift'] = trades.time_until_end.shift(-1).diff()
trades.loc[len(trades) - 1, 'time_diff_shift'] = 1
trades['fee_in_usd'] = trades.fee * trades.base_asset_price
trades['fee_in_bps'] = trades.fee / trades.out_without_fee * 100 * 100
x_liquidity_usd = trades.x_reserves * trades.base_asset_price
y_liquidity_usd = trades.y_reserves * trades.base_asset_price * trades.spot_price
trades['total_liquidity_usd'] = x_liquidity_usd + y_liquidity_usd
trades['trade_volume_usd'] = trades.out_with_fee * trades.base_asset_price
model_numbers = []
for name in trades['model_name']:
    if name == 'Element':
        model_numbers.append(0)
    elif name == 'YieldSpacev2':
        model_numbers.append(1)
    elif name == 'YieldSpacev2MinFee':
        model_numbers.append(2)
trades['model_number'] = model_numbers

# pr is the percent change in spot price since day 1
trades['pr'] = trades.loc[:, 'spot_price'] - trades.loc[0, 'spot_price'] # this is APR (does not include compounding)
# pu takes that percent change and normalizes it to be equal to init_price_per_share at the beginning, so you can compare its progression vs. price_per_share
trades['pu'] = (trades.loc[:, 'pr'] + 1) * trades.loc[:, 'init_price_per_share'] # this is APR (does not include compounding)

# create explicit column that increments per trade
trades = trades.reset_index().rename(columns = {'index': 'trade_number'})


preserved_columns = [
    'day',
    'model_name',
    'model_number',
    'init_vault_age',
    'pu',
]
trades_reduced = trades.groupby(preserved_columns).agg({
    'trade_volume_usd': ['sum'],
    'fee_in_usd': ['mean', 'sum'],
    'fee_in_bps': ['mean', 'sum'],
    'pool_apy': ['mean'],
    'vault_apy': ['mean'],
})
trades_reduced.columns = ['_'.join(col).strip() for col in trades_reduced.columns.values]
trades_reduced = trades_reduced.reset_index()
#trades_reduced['spot_price_growth'] = (trades_reduced.groupby(['day'])['pu'].apply(pd.Series.pct_change) + 1)
#trades_reduced['spot_price_growth_sum'] = trades_reduced.groupby(['day']).agg({'spot_price_growth':['sum']})

trades_reduced

In [ ]:
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = trades_reduced.fee_in_usd_sum,
                   #colorscale = 'Electric',
                   colorscale = 'viridis',
                   #colorscale = 'tealrose',
                   showscale = True,
                   cmin = trades_reduced.fee_in_usd_sum.min(),
                   cmax = trades_reduced.fee_in_usd_sum.max()),
        dimensions = list([
            dict(tickvals = [0, 1, 2],
                 ticktext = ['Element', 'YieldSpacev2', 'YieldSpacev2MinFee'],
                 label = 'Model Type',
                 values = trades_reduced.model_number),
            dict(range = [trades_reduced.init_vault_age.min(), trades_reduced.init_vault_age.max()],
                 #tickvals = list(range(trades_reduced.init_vault_age.min(), trades_reduced.init_vault_age.max()+2, 5)),
                 label = 'Initial Vault Age',
                 values = trades_reduced.init_vault_age),
            dict(range = [trades_reduced.vault_apy_mean.min(), trades_reduced.vault_apy_mean.max()],
                 label = "Vault APY",
                 values = trades_reduced.vault_apy_mean),
            dict(range = [trades_reduced.pool_apy_mean.min(), trades_reduced.pool_apy_mean.max()],
                 label = 'Mean Pool APY',
                 values = trades_reduced.pool_apy_mean),
            dict(range = [trades_reduced.fee_in_usd_sum.min(), trades_reduced.fee_in_usd_sum.max()],
                 label = 'Fee (USD)',
                 values = trades_reduced.fee_in_usd_sum),
        ])
    )
)

fig.show()